# 📂 문서 군집화 (Document Clustering): 뒤죽박죽 문서 정리하기

안녕! 이번에는 **'답이 없는 문제'**를 풀어볼 거야. 
이전까지는 "이건 사과야, 이건 배야"라고 알려주는(지도 학습) 방식이었다면, 
이번에는 **"그냥 이거 다 섞여 있는데, 비슷한 것끼리 묶어봐!"** 라고 시키는 **비지도 학습(Unsupervised Learning)**, 그 중에서도 **군집화(Clustering)**를 배울 거야.

> **비유**: 빨래 바구니에 양말, 티셔츠, 바지가 마구 섞여 있어. 이걸 '양말 더미', '티셔츠 더미', '바지 더미'로 나누는 작업이라고 생각하면 돼!

---

## 1. 데이터 불러오기: 흩어지 파일 모으기 🗂️

이번에 쓸 데이터는 호텔, 자동차, 전자기기 등에 대한 **리뷰 데이터**야. 
그런데 파일이 하나로 예쁘게 정리된 게 아니라, 주제별로 51개의 파일이 흩어져 있어. 이걸 하나로 모으는 것부터 시작하자.

### 🛠️ 파일 다루는 도구들
- `os`: 윈도우/맥 같은 운영체제 기능을 빌려 쓰는 도구 (폴더 경로 찾기 등)
- `glob`: "*.txt" 처럼 특정 패턴을 가진 파일들을 몽땅 찾아주는 도구

In [1]:
import pandas as pd
import glob, os

# 1. 데이터가 있는 폴더 경로 설정 (네 컴퓨터 환경에 맞게 경로가 잘 설정되어 있는지 확인!)
path = './data/OpinosisDataset1.0/topics'

# 2. 해당 폴더 안에 있는 모든 .data 파일(*.data)의 이름을 리스트로 가져오기
all_files = glob.glob(os.path.join(path, '*.data'))

print(f"발견된 파일 개수: {len(all_files)}개")
print("첫 3개 파일만 볼까?", all_files[:3])

발견된 파일 개수: 51개
첫 3개 파일만 볼까? ['./data/OpinosisDataset1.0/topics/battery-life_ipod_nano_8gb.txt.data', './data/OpinosisDataset1.0/topics/gas_mileage_toyota_camry_2007.txt.data', './data/OpinosisDataset1.0/topics/room_holiday_inn_london.txt.data']


### 📝 파일 읽어서 표(DataFrame)로 만들기
이제 찾아낸 파일들을 하나씩 읽어서, **(파일명, 리뷰내용)** 형태의 표로 정리할 거야.

In [2]:
filename_list = []  # 파일 제목을 저장할 리스트
opinion_text = []   # 파일 내용을 저장할 리스트

for file in all_files:
    # 1. 파일 읽기 (Latin-1 인코딩 사용)
    df = pd.read_table(file, index_col=None, header=None, encoding='latin1')
    
    # 2. 파일명 예쁘게 다듬기 (경로 제거, 확장자 제거)
    # 예: './data/.../battery_life.data' -> 'battery_life'
    filename_ = file.split('/')[-1] # 경로 구분자(/)로 자르고 맨 뒤에꺼(파일명)만 가져오기
    filename = filename_.split('.')[0] # 점(.)으로 자르고 앞에꺼(확장자 제외)만 가져오기
    filename_list.append(filename)

    # 3. 파일 내용을 하나의 문자열로 합치기
    opinion_text.append(df.to_string(header=False, index=False))

# 4. DataFrame으로 변환
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head() # 상위 5개만 확인

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,gas_mileage_toyota_camry_2007,...
2,room_holiday_inn_london,...
3,location_holiday_inn_london,...
4,staff_bestwestern_hotel_sfo,...


---

## 2. 전처리 및 벡터화: 컴퓨터가 읽기 편하게 🧹

이전에 배웠던 **TF-IDF**를 사용할 거야. 
이번에는 조금 더 고급 기술을 추가해보자.

### 🔩 어근 추출 (Lemmatization)
- `running`, `runs`, `ran` -> 모두 `run`으로 통일하는 작업이야.
- 단어의 '뿌리'를 찾아주면 컴퓨터가 더 정확하게 의미를 파악할 수 있어.

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# NLTK 데이터가 없으면 다운로드 (한번만 하면 됨)
# nltk.download('punkt')
# nltk.download('wordnet')

# 나만의 토큰화 함수 만들기 (특수문자 제거 + 어근 추출)
def LemNormalize(text):
    # 1. 소문자 변환
    text = text.lower()
    
    # 2. 특수문자 제거 (딕셔너리 활용)
    # ord(): 문자를 숫자로 변환. 예: '!' -> 33
    # remove_punct_dict = {33: None, ...} 이런 식이야. 특수문자를 만나면 None(삭제)으로 바꿔라!
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    text = text.translate(remove_punct_dict)
    
    # 3. 단어 토큰화
    tokens = nltk.word_tokenize(text)
    
    # 4. 어근 추출 (동사 기준)
    lemmar = WordNetLemmatizer()
    return [lemmar.lemmatize(token, pos='v') for token in tokens]

[nltk_data] Downloading package punkt to /Users/gimdabin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gimdabin/nltk_data...


### 🔢 TF-IDF 설정하기
- `tokenizer`: 방금 만든 어근 추출 함수를 사용.
- `stop_words`: 영어 불용어 제거.
- `ngram_range`: (1, 2) -> 단어 1개짜리와 2개짜리 묶음(Bi-gram)을 모두 사용.
- `min_df=0.05`, `max_df=0.85`: 
    - 너무 희귀한 단어(5% 미만 등장)는 버리고,
    - 너무 흔한 단어(85% 이상 등장)도 버려서 핵심 단어만 남기기!

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(
    tokenizer=LemNormalize, 
    stop_words='english', 
    ngram_range=(1,2), 
    min_df=0.05,
    max_df=0.85
)

# 학습 및 변환 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

print("벡터화 완료! 행렬 크기:", feature_vect.shape)
# (문서 51개, 단어 약 4000여개)의 행렬이 만들어졌을 거야.

/opt/anaconda3/envs/nlp_env/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/envs/nlp_env/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:411: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  warnings.warn(


벡터화 완료! 행렬 크기: (51, 4072)


---

## 3. K-Means 군집화: 끼리끼리 뭉쳐라! 👨‍👩‍👧‍👦

> **"모르는 사람들 51명을 강당에 풀어놓고, 성향이 비슷한 5개 그룹으로 알아서 모이라고 하는 것"**

**K-Means 알고리즘**은 가장 대표적인 군집화 방법이야.
1. 임의의 중심점(Centroid) K개를 찍어.
2. 각 데이터는 가장 가까운 중심점 팀으로 들어가.
3. 팀원들의 평균 위치로 중심점을 이동시켜.
4. 중심점이 더 이상 안 움직일 때까지 2~3번 반복!

우리는 주제가 대충 5개 정도(호텔, 자동차, 전자기기 등) 될 거라고 예상하고 **K=5**로 설정해볼게.

In [5]:
from sklearn.cluster import KMeans

# 1. 모델 생성 (군집 5개)
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)

# 2. 학습 및 예측
# feature_vect(TF-IDF 데이터)를 넣으면, 각 문서가 0, 1, 2, 3, 4 중 어느 그룹인지 알려줌
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_ # 각 문서의 그룹 번호
cluster_centers = km_cluster.cluster_centers_ # 각 그룹의 중심점 좌표

# 3. 결과 저장
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,1
1,gas_mileage_toyota_camry_2007,...,4
2,room_holiday_inn_london,...,3
3,location_holiday_inn_london,...,0
4,staff_bestwestern_hotel_sfo,...,3


### 📊 결과 확인: 잘 묶였을까?
각 그룹(Cluster)별로 어떤 문서들이 모였는지 확인해보자.
파일 이름을 보면 대충 비슷한 것끼리 모였는지 알 수 있겠지?

In [6]:
# 그룹별로 출력해서 확인해보기
for i in range(5):
    print(f"\n####### Cluster {i} #######")
    # 해당 그룹에 속한 문서들의 파일명만 뽑아서 출력
    print(document_df[document_df['cluster_label']==i]['filename'].sort_values().tolist())


####### Cluster 0 #######
['food_holiday_inn_london', 'food_swissotel_chicago', 'location_holiday_inn_london', 'price_amazon_kindle', 'price_holiday_inn_london', 'service_bestwestern_hotel_sfo', 'service_holiday_inn_london', 'service_swissotel_hotel_chicago']

####### Cluster 1 #######
['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps', 'eyesight-issues_amazon_kindle', 'features_windows7', 'fonts_amazon_kindle', 'keyboard_netbook_1005ha', 'navigation_amazon_kindle', 'performance_netbook_1005ha', 'satellite_garmin_nuvi_255W_gps', 'screen_garmin_nuvi_255W_gps', 'screen_ipod_nano_8gb', 'screen_netbook_1005ha', 'size_asus_netbook_1005ha', 'sound_ipod_nano_8gb', 'speed_garmin_nuvi_255W_gps', 'speed_windows7', 'updates_garmin_nuvi_255W_gps', 'video_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps']

####### Cluster 2 #######
['interi

### 🕵️‍♂️ 결과 해석
실행 결과를 보면 놀랍게도 비슷한 주제끼리 모인 걸 볼 수 있을 거야.
- 어떤 그룹은 **자동차** 관련(Toyota, Honda...)
- 어떤 그룹은 **호텔** 관련(Hotel, Inn, Room...)
- 어떤 그룹은 **전자기기** 관련(iPod, Kindle...)

컴퓨터는 단어들의 **등장 패턴(통계)**만 보고 이렇게 주제를 분류해낸 거야. 신기하지? 😎

---

## 🎓 마무리 퀴즈

**Q1. 정답을 알려주지 않고 데이터의 특징만으로 비슷한 것끼리 묶는 학습 방식을 무엇이라 할까?**
1. 지도 학습 (Supervised Learning)
2. 비지도 학습 (Unsupervised Learning)
3. 강화 학습 (Reinforcement Learning)

**Q2. TF-IDF 파라미터 중, 문서에 '너무 자주(예: 85% 이상)' 등장하는 단어를 배제하는 옵션은?**
1. min_df
2. max_df
3. ngram_range

**정답**

<details>
<summary>클릭해서 확인!</summary>

1. **2번 비지도 학습** (선생님 없이 스스로 공부하기!)
2. **2번 max_df** (너무 흔하면 특징이 안 되니까 위에서 잘라냄!)

</details>